# Housing Model Training Pipeline + Hyperparameter Tuners

## 1. Import Libraries
Import all necessary libraries for data processing, transformation, modeling, and evaluation.

In [4]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

## 2. Define Custom Transformer
The `ClusterSimilarity` transformer creates geographic cluster features using KMeans clustering.

In [5]:
class ClusterSimilarity(BaseEstimator, TransformerMixin):
    """
    Creates features based on similarity to geographic clusters
    
    Parameters:
    n_clusters : Number of clusters to create
    gamma : Controls the influence radius of clusters
    random_state : Random seed for reproducibility
    """
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state
    
    def fit(self, X, y=None, sample_weight=None):
        if hasattr(X, "values"):
            X = X.values
        self.kmeans_ = KMeans(n_clusters=self.n_clusters, 
                             random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self
    
    def transform(self, X):
        if hasattr(X, "values"):
            X = X.values
        distances = np.linalg.norm(X[:, np.newaxis] - self.kmeans_.cluster_centers_, 
                                 axis=2)
        return np.exp(-self.gamma * distances ** 2)
    
    def get_params(self, deep=True):
        return {"n_clusters": self.n_clusters, 
                "gamma": self.gamma, 
                "random_state": self.random_state}
    
    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        return self

## 3. Load and Prepare Data
Load the housing dataset and create stratified train/test splits based on income categories.

In [6]:
def load_housing_data():
    """Load California housing dataset from remote URL"""
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

# Load and prepare data
housing = load_housing_data()

# Create income categories for stratified sampling
housing["income_cat"] = pd.cut(
    housing["median_income"],
    bins=[0.0, 1.5, 3.0, 4.5, 6.0, np.inf],
    labels=[1, 2, 3, 4, 5]
)

# Split into stratified train/test sets
strat_train_set, strat_test_set = train_test_split(
    housing,
    test_size=0.2,
    stratify=housing["income_cat"],
    random_state=42
)

# Prepare training data
housing = strat_train_set.drop(["median_house_value", "income_cat"], axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

## 4. Define Transformation Pipelines
Create custom transformation pipelines for different feature types including ratio features, log transformations, and categorical encoding.

In [7]:
# Helper functions for ratio features
def column_ratio(X):
    """Calculate ratio between two columns"""
    return X[:, [0]] / X[:, [1]]

def ratio_name(function_transformer, feature_names_in):
    """Generate feature name for ratio features"""
    return ["ratio"]

# Pipeline for ratio features
def ratio_pipeline():
    """Pipeline for ratio features: Impute → Calculate ratio → Scale"""
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_ratio, feature_names_out=ratio_name),
        StandardScaler()
    )

# Pipeline for log-transformed features
log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler()
)

# Pipeline for categorical features
cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore")
)

# Default pipeline for numeric features
default_num_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler()
)

# Initialize geographic similarity transformer
cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1.0, random_state=42)

## 5. Create Column Transformer
Combine all pipelines into a single preprocessing pipeline using ColumnTransformer.

In [8]:
preprocessing = ColumnTransformer([
    # Bedrooms to rooms ratio
    ("bedrooms", ratio_pipeline(), ["total_bedrooms", "total_rooms"]),
    
    # Rooms per household
    ("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]),
    
    # Population per household
    ("people_per_house", ratio_pipeline(), ["population", "households"]),
    
    # Log-transformed features
    ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population",
                         "households", "median_income"]),
    
    # Geographic cluster similarity
    ("geo", cluster_simil, ["latitude", "longitude"]),
    
    # Categorical feature (ocean proximity)
    ("cat", cat_pipeline, ["ocean_proximity"])
], 
# Handle remaining numeric features
remainder=default_num_pipeline)

## 6. Apply Preprocessing
Transform the training data and verify the output feature matrix.

In [9]:
# Apply the full preprocessing pipeline
housing_prepared = preprocessing.fit_transform(housing)

# Display resulting feature matrix shape
print("Preprocessed data shape:", housing_prepared.shape)
print("\nExpected features breakdown:")
print("  - 3 ratio features")
print("  - 5 log-transformed features")
print("  - 10 geographic similarity features")
print("  - 5 one-hot encoded categories")
print("  - 1 remaining numeric feature")
print("Total: 24 features")

Preprocessed data shape: (16512, 24)

Expected features breakdown:
  - 3 ratio features
  - 5 log-transformed features
  - 10 geographic similarity features
  - 5 one-hot encoded categories
  - 1 remaining numeric feature
Total: 24 features


---
# **Exercise**
* Try replacing the GridSearchCV with a RandomizedSearchCV.

### Step 1: **Subset the First 5,000 Training Instances**

**Subset for Speed**
* SVR models are computationally expensive. We use only the first 5,000 rows for cross-validation experiments.


In [10]:
# Limit training set to 5,000 instances for SVR (speed)
X_small = housing_prepared[:5000]
y_small = housing_labels[:5000]

### Step 2: **Define Distributions for RandomizedSearchCV**
**Define Distributions for Hyperparameters**
* We use log-uniform distributions for `C` and `gamma` to explore a wide, exponentially scaled range of values.

In [11]:
from scipy.stats import loguniform

# Distributions to sample from
param_distributions = {
    "kernel": ["linear", "rbf"],
    "C": loguniform(0.01, 100),
    "gamma": loguniform(0.001, 1)  # only relevant for 'rbf'
}

### Step 3: **Run RandomizedSearchCV on SVR**
**Run RandomizedSearchCV**
* We sample 20 random combinations of hyperparameters and train using 3-fold CV. This is more efficient than exhaustive grid search.

In [12]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR

svr = SVR()

random_search = RandomizedSearchCV(
    estimator=svr,
    param_distributions=param_distributions,
    n_iter=20,  # total combinations to try
    scoring="neg_root_mean_squared_error",
    cv=3,
    random_state=42,
    verbose=2,
    n_jobs=-1
)

random_search.fit(X_small, y_small)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


,estimator,SVR()
,param_distributions,"{'C': <scipy.stats....002408B2479E0>, 'gamma': <scipy.stats....002408AC89AC0>, 'kernel': ['linear', 'rbf']}"
,n_iter,20
,scoring,'neg_root_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,3
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


### Step 4: **View the Best Parameters and Validation RMSE**
**Evaluate Best Model on Validation Set**
* We inspect which sampled parameter set performed best and what its average RMSE was across the CV folds.


In [17]:
best_params = random_search.best_params_
best_rmse = -random_search.best_score_

print("✅ Best Parameters:", best_params)
print("📉 Best Validation RMSE:", best_rmse)

✅ Best Parameters: {'C': np.float64(85.68869785189007), 'gamma': np.float64(0.025135566617708285), 'kernel': 'linear'}
📉 Best Validation RMSE: 82193.60116675713


### Step 5: **Evaluate on Full Test Set**
**Evaluate on Unseen Data**
**Evaluate on Test Set**
* We assess the real-world performance of the best model on unseen data by computing the test set RMSE.

In [18]:
# Prepare test set
X_test = strat_test_set.drop(["median_house_value", "income_cat"], axis=1)
y_test = strat_test_set["median_house_value"].copy()
X_test_prepared = preprocessing.transform(X_test)

# Predict using the best model
final_model = random_search.best_estimator_
y_pred = final_model.predict(X_test_prepared)

# Manually compute RMSE
from sklearn.metrics import mean_squared_error
import numpy as np

test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Final Test Set RMSE:", test_rmse)

Final Test Set RMSE: 100011.7167113282
